In [1]:
from pandas_profiling import ProfileReport
import pandas as pd

In [39]:
df_accidents = pd.read_csv('../Road Safety Data/dft-road-casualty-statistics-accident-2020.csv')
df_casualties = pd.read_csv('../Road Safety Data/dft-road-casualty-statistics-casualty-2020.csv')
df_vehicles = pd.read_csv('../Road Safety Data/dft-road-casualty-statistics-vehicle-2020.csv')

C:\Python38\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.



In [40]:
# accidents = ProfileReport(df_accidents)
# accidents.to_file(output_file='Data reports/accidents.html')
# casualties = ProfileReport(df_casualties)
# casualties.to_file(output_file='Data reports/casualties.html')
# vehicles = ProfileReport(df_vehicles)
# vehicles.to_file(output_file='Data reports/vehicles.html')

In [41]:
casualties = df_casualties[['accident_reference', 'casualty_severity', 'vehicle_reference', 'casualty_reference', 'sex_of_casualty', 'age_band_of_casualty', 'casualty_home_area_type']]

In [42]:
accidents = df_accidents[['accident_reference', 'longitude', 'latitude']]

In [43]:
vehicles = df_vehicles[['accident_reference', 'vehicle_reference', 'vehicle_type', 'towing_and_articulation', 'vehicle_left_hand_drive', 'journey_purpose_of_driver', 'engine_capacity_cc', 'propulsion_code', 'age_of_vehicle', 'generic_make_model']]

In [44]:
casualties

,accident_reference,casualty_severity,vehicle_reference,casualty_reference,sex_of_casualty,age_band_of_casualty,casualty_home_area_type
0,010219808,3,1,1,1,6,1
1,010220496,3,1,1,2,1,1
2,010220496,3,1,2,2,1,1
3,010228005,3,1,1,1,5,1
4,010228006,2,1,1,1,8,1
...,...,...,...,...,...,...,...
115579,991027064,2,2,1,1,3,1
115580,991029573,3,1,1,2,9,1
115581,991030297,2,2,1,1,7,2
115582,991030900,3,2,1,1,11,1


In [45]:
c_and_a = pd.merge(casualties, accidents, on=['accident_reference'])

In [46]:
all_data = pd.merge(c_and_a, vehicles, on=['accident_reference', 'vehicle_reference'])

In [47]:
all_data = all_data.dropna()

In [48]:
all_data = all_data[(all_data['sex_of_casualty'] != -1) & (all_data['sex_of_casualty'] != 9)] #delete missing

In [49]:
all_data = all_data[all_data['age_band_of_casualty'] != -1] #delete missing

In [50]:
all_data = all_data[all_data['casualty_home_area_type'] != -1] #delete missing

In [51]:
all_data = all_data[all_data['vehicle_type'] != -1] #delete missing

In [52]:
all_data = all_data[(all_data['towing_and_articulation'] != -1) & (all_data['towing_and_articulation'] != 9)] #dropping missing

In [53]:
all_data = all_data[(all_data['vehicle_left_hand_drive'] != -1) & (all_data['vehicle_left_hand_drive'] != 9)] #dropping missing

In [54]:
all_data = all_data[(all_data['journey_purpose_of_driver'] != -1)] #delete missing

In [55]:
all_data = all_data[(all_data['generic_make_model'] != '-1')] #delete missing

In [56]:
vehicle_means = all_data.groupby('vehicle_type')['engine_capacity_cc'].mean()

In [57]:
for i in range(len(all_data)):
    if all_data['engine_capacity_cc'].iloc[i] == -1:
        all_data['engine_capacity_cc'].iloc[i] = vehicle_means[all_data.iloc[i].vehicle_type]
    

In [58]:
all_data = all_data[all_data['engine_capacity_cc'] != -1] #dropping rows/vehicles that have no engine capacity e.g. horse, cycle

In [59]:
all_data = all_data[all_data['propulsion_code'] != -1] #dropping missing

In [60]:
all_data = all_data[all_data['age_of_vehicle'] != -1] #dropping missing

In [83]:
severity_dict = {
    1: 'Fatal',
    2: 'Serious',
    3: 'Slight'
}

severity_dict_reverse = {
    1: 3,
    2: 2,
    3: 1
}

all_data['casualty_severity_name'] = all_data['casualty_severity'].replace(severity_dict)
all_data['casualty_severity'] = all_data['casualty_severity'].replace(severity_dict_reverse)

In [62]:
sex_dict = {
    1: 'Male',
    2: 'Female'
}

all_data['sex_of_casualty'] = all_data['sex_of_casualty'].replace(sex_dict)

In [63]:
age_dict = {
    1:	'0 - 5',
    2: '6 - 10',
    3: '11 - 15',
    4: '16 - 20',
    5: '21 - 25',
    6: '26 - 35',
    7: '36 - 45',
    8: '46 - 55',
    9: '56 - 65',
    10: '66 - 75',
    11: '75+',
}

all_data['age_band_of_casualty'] = all_data['age_band_of_casualty'].replace(age_dict)

In [64]:
home_dict = {
    1: 'Urban area', 
    2: 'Small town', 
    3: 'Rural', 
}

all_data['casualty_home_area_type'] = all_data['casualty_home_area_type'].replace(home_dict)

In [65]:
vehicle_dict = {
    1: 'Pedal cycle', 
    2: 'Motorcycle 50cc and under', 
    3: 'Motorcycle 125cc and under', 
    4: 'Motorcycle over 125cc and up to 500cc', 
    5: 'Motorcycle over 500cc', 
    8: 'Taxi/Private hire car', 
    9: 'Car', 
    10: 'Minibus (8 - 16 passenger seats)', 
    11: 'Bus or coach (17 or more pass seats)', 
    16: 'Ridden horse', 
    17: 'Agricultural vehicle', 
    18: 'Tram', 
    19: 'Van / Goods 3.5 tonnes mgw or under', 
    20: 'Goods over 3.5t. and under 7.5t', 
    21: 'Goods 7.5 tonnes mgw and over', 
    22: 'Mobility scooter', 
    23: 'Electric motorcycle', 
    90: 'Other vehicle', 
    97: 'Motorcycle - unknown cc', 
    98: 'Goods vehicle - unknown weight', 
    99: 'Unknown vehicle type (self rep only)', 
    103: 'Motorcycle - Scooter (1979-1998)', 
    104: 'Motorcycle (1979-1998)', 
    105: 'Motorcycle - Combination (1979-1998)', 
    106: 'Motorcycle over 125cc (1999-2004)', 
    108: 'Taxi (excluding private hire cars) (1979-2004)', 
    109: 'Car (including private hire cars) (1979-2004)', 
    110: 'Minibus/Motor caravan (1979-1998)', 
    113: 'Goods over 3.5 tonnes (1979-1998)', 
}

all_data['vehicle_type'] = all_data['vehicle_type'].replace(vehicle_dict)

In [66]:
towing_dict = {
    0: 'No tow/articulation',  
    1: 'Articulated vehicle', 
    2: 'Double or multiple trailer', 
    3: 'Caravan',
    4: 'Single trailer',
    5: 'Other tow',
}

all_data['towing_and_articulation'] = all_data['towing_and_articulation'].replace(towing_dict)

In [67]:
left_dict = {
    1: 'No',
    2: 'Yes',
}

all_data['vehicle_left_hand_drive'] = all_data['vehicle_left_hand_drive'].replace(left_dict)

In [68]:
journey_dict = {
    1: 'Journey as part of work',
    2: 'Commuting to/from work',
    3: 'Taking pupil to/from school',
    4: 'Pupil riding to/from school',
    5: 'Other',
    6: 'Not known',
}

all_data['journey_purpose_of_driver'] = all_data['journey_purpose_of_driver'].replace(journey_dict)

In [69]:
propulsion_dict = {
    1: 'Petrol',
    2: 'Heavy oil',
    3: 'Electric',
    4: 'Steam',
    5: 'Gas',
    6: 'Petrol/Gas (LPG)',
    7: 'Gas/Bi-fuel',
    8: 'Hybrid electric',
    9: 'Gas Diesel',
    10: 'New fuel technology',
    11: 'Fuel cells',
    12: 'Electric diesel',
}

all_data['propulsion_code'] = all_data['propulsion_code'].replace(propulsion_dict)

In [70]:
all_data['age_of_vehicle'] = all_data.groupby(['vehicle_type'])['age_of_vehicle'].transform(lambda x: pd.qcut(x, q=5, precision=0, duplicates='drop'))

In [71]:
all_data['age_of_vehicle'] = all_data['age_of_vehicle'].astype('str').str.replace('-1.0', '0.0')

C:\Users\20172010\AppData\Local\Temp\ipykernel_1716\1925975203.py:1: FutureWarning:

The default value of regex will change from True to False in a future version.



In [72]:
all_data['engine_capacity_cc'] = all_data.groupby(['vehicle_type'])['engine_capacity_cc'].transform(lambda x: pd.cut(x, bins=3, precision=0, duplicates='drop'))

In [73]:
all_data['engine_capacity_cc'] = all_data['engine_capacity_cc'].astype('str').str.replace('-1.0', '0.0')

C:\Users\20172010\AppData\Local\Temp\ipykernel_1716\856273390.py:1: FutureWarning:

The default value of regex will change from True to False in a future version.



In [74]:
all_data

,accident_reference,casualty_severity,vehicle_reference,casualty_reference,sex_of_casualty,age_band_of_casualty,casualty_home_area_type,longitude,latitude,vehicle_type,towing_and_articulation,vehicle_left_hand_drive,journey_purpose_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,casualty_severity_name
1,010278556,2,1,1,Male,16 - 20,Urban area,-0.145519,51.546549,Motorcycle 125cc and under,No tow/articulation,No,Not known,"(48.0, 432.0]",Petrol,"(3.0, 5.0]",HONDA CBR125R,Serious
2,010278558,3,1,1,Male,36 - 45,Urban area,-0.066682,51.497938,Motorcycle 125cc and under,No tow/articulation,No,Journey as part of work,"(48.0, 432.0]",Petrol,"(0.0, 1.0]",YAMAHA GPD 125,Slight
4,010278561,2,1,1,Male,36 - 45,Urban area,-0.055243,51.546633,Bus or coach (17 or more pass seats),No tow/articulation,No,Journey as part of work,"(3891.0, 6933.0]",Heavy oil,"(0.0, 3.0]",ALEXANDER DENNIS MODEL MISSING,Serious
5,010278564,3,1,1,Female,21 - 25,Urban area,-0.265137,51.416124,Motorcycle 125cc and under,No tow/articulation,No,Not known,"(48.0, 432.0]",Petrol,"(5.0, 9.0]",HONDA WW125EX2,Slight
6,010278570,3,1,1,Male,26 - 35,Urban area,-0.363697,51.569271,Car,No tow/articulation,No,Commuting to/from work,"(586.0, 5034.0]",Petrol,"(0.0, 4.0]",TOYOTA AYGO,Slight
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92442,990963763,3,1,1,Female,56 - 65,Urban area,-3.287198,55.943143,Car,No tow/articulation,No,Not known,"(586.0, 5034.0]",Petrol,"(0.0, 4.0]",HYUNDAI I10,Slight
92443,990963763,3,2,2,Female,26 - 35,Urban area,-3.287198,55.943143,Car,No tow/articulation,No,Not known,"(586.0, 5034.0]",Petrol,"(0.0, 4.0]",KIA CEED,Slight
92444,990963772,3,2,1,Female,56 - 65,Urban area,-2.993709,56.465003,Car,No tow/articulation,No,Other,"(586.0, 5034.0]",Petrol,"(0.0, 4.0]",VOLKSWAGEN T-ROC,Slight
92445,990963785,2,1,1,Male,46 - 55,Urban area,-2.393835,57.015628,Motorcycle over 500cc,No tow/articulation,No,Other,"(398.0, 1086.0]",Petrol,"(14.0, 20.0]",YAMAHA YZF R1,Serious


In [339]:
# all_data.to_csv('processed_data_2020.csv', index=False)

In [143]:
import plotly.express as px
fig = px.histogram(all_data, x="Cluster Labels", y='casualty_severity', histfunc='sum')
fig.show()

In [112]:
from kmodes.kmodes import KModes


In [141]:
model = KModes(n_jobs = -1, n_clusters = 10, init = 'Huang', random_state = 0).fit(all_data.drop(columns=['accident_reference', 'casualty_reference', 'vehicle_reference', 'casualty_severity']))

In [142]:
all_data['Cluster Labels'] = model.labels_

In [145]:
all_data

,accident_reference,casualty_severity,vehicle_reference,casualty_reference,sex_of_casualty,age_band_of_casualty,casualty_home_area_type,longitude,latitude,vehicle_type,towing_and_articulation,vehicle_left_hand_drive,journey_purpose_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,casualty_severity_name,Cluster Labels
1,010278556,2,1,1,Male,16 - 20,Urban area,-0.145519,51.546549,Motorcycle 125cc and under,No tow/articulation,No,Not known,"(48.0, 432.0]",Petrol,"(3.0, 5.0]",HONDA CBR125R,Serious,0
2,010278558,1,1,1,Male,36 - 45,Urban area,-0.066682,51.497938,Motorcycle 125cc and under,No tow/articulation,No,Journey as part of work,"(48.0, 432.0]",Petrol,"(0.0, 1.0]",YAMAHA GPD 125,Slight,3
4,010278561,2,1,1,Male,36 - 45,Urban area,-0.055243,51.546633,Bus or coach (17 or more pass seats),No tow/articulation,No,Journey as part of work,"(3891.0, 6933.0]",Heavy oil,"(0.0, 3.0]",ALEXANDER DENNIS MODEL MISSING,Serious,0
5,010278564,1,1,1,Female,21 - 25,Urban area,-0.265137,51.416124,Motorcycle 125cc and under,No tow/articulation,No,Not known,"(48.0, 432.0]",Petrol,"(5.0, 9.0]",HONDA WW125EX2,Slight,5
6,010278570,1,1,1,Male,26 - 35,Urban area,-0.363697,51.569271,Car,No tow/articulation,No,Commuting to/from work,"(586.0, 5034.0]",Petrol,"(0.0, 4.0]",TOYOTA AYGO,Slight,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92442,990963763,1,1,1,Female,56 - 65,Urban area,-3.287198,55.943143,Car,No tow/articulation,No,Not known,"(586.0, 5034.0]",Petrol,"(0.0, 4.0]",HYUNDAI I10,Slight,5
92443,990963763,1,2,2,Female,26 - 35,Urban area,-3.287198,55.943143,Car,No tow/articulation,No,Not known,"(586.0, 5034.0]",Petrol,"(0.0, 4.0]",KIA CEED,Slight,5
92444,990963772,1,2,1,Female,56 - 65,Urban area,-2.993709,56.465003,Car,No tow/articulation,No,Other,"(586.0, 5034.0]",Petrol,"(0.0, 4.0]",VOLKSWAGEN T-ROC,Slight,4
92445,990963785,2,1,1,Male,46 - 55,Urban area,-2.393835,57.015628,Motorcycle over 500cc,No tow/articulation,No,Other,"(398.0, 1086.0]",Petrol,"(14.0, 20.0]",YAMAHA YZF R1,Serious,4


In [166]:
all_data.groupby('Cluster Labels')['vehicle_type'].value_counts().unstack()

vehicle_type,Agricultural vehicle,Bus or coach (17 or more pass seats),Car,Goods 7.5 tonnes mgw and over,Goods over 3.5t. and under 7.5t,Goods vehicle - unknown weight,Minibus (8 - 16 passenger seats),Motorcycle 125cc and under,Motorcycle 50cc and under,Motorcycle over 125cc and up to 500cc,Motorcycle over 500cc,Other vehicle,Taxi/Private hire car,Van / Goods 3.5 tonnes mgw or under
Cluster Labels,,,,,,,,,,,,,,
0,10.0,88.0,3437.0,54.0,19.0,12.0,4.0,565.0,29.0,107.0,613.0,3.0,57.0,342.0
1,19.0,405.0,12751.0,190.0,59.0,46.0,15.0,10.0,NaN,1.0,9.0,29.0,339.0,1578.0
2,8.0,89.0,2185.0,46.0,13.0,8.0,7.0,NaN,NaN,NaN,1.0,13.0,22.0,253.0
3,2.0,71.0,7600.0,49.0,7.0,13.0,2.0,959.0,55.0,139.0,594.0,7.0,163.0,415.0
4,1.0,13.0,3533.0,9.0,2.0,9.0,NaN,211.0,9.0,37.0,372.0,NaN,11.0,50.0
5,NaN,43.0,11363.0,2.0,NaN,3.0,1.0,165.0,26.0,15.0,73.0,NaN,51.0,25.0
6,1.0,4.0,1237.0,NaN,2.0,NaN,NaN,190.0,38.0,9.0,16.0,1.0,3.0,7.0
7,NaN,NaN,3980.0,NaN,NaN,NaN,NaN,304.0,84.0,34.0,9.0,NaN,37.0,NaN
8,1.0,2.0,1243.0,5.0,3.0,3.0,NaN,16.0,1.0,2.0,32.0,2.0,7.0,29.0


In [182]:
import plotly.express as px

fig = px.imshow(all_data.groupby('Cluster Labels')['casualty_home_area_type'].value_counts().unstack())
fig.show()

In [183]:
import plotly.express as px
df = px.data.iris()
fig = px.parallel_coordinates(all_data.groupby('Cluster Labels')['casualty_home_area_type'].value_counts().unstack(), color=all_data['Cluster Labels'].unique())
fig.show()

In [ ]:
all_data['engine_capacity_cc']